# 导入需要的模块

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter

# 构建神经网络

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        self.bn = nn.BatchNorm2d(20)
        
    def forward(self,x):
        x = F.max_pool2d(self.conv1(x),2)
        x = F.relu(x)+F.relu(-x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = self.bn(x)
        
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = self.fc2(x)
        x = F.softmax(x,dim=1)
        return x

# 将模型保存为graph

In [15]:
writer = SummaryWriter(log_dir=r'E:\Notebook\TensorboardX可视化处理\logs')

In [16]:
input = torch.rand(32,1,28,28)

In [17]:
model = Net()

In [19]:
writer.add_graph(model,input)

# 用TensorboardX可视化损失值

In [20]:
dtype = torch.FloatTensor

In [21]:
writer = SummaryWriter(log_dir='logs/',comment='Linear')

In [22]:
np.random.seed(100)

In [23]:
import numpy as np

In [24]:
x_train = np.linspace(-1,1,100).reshape(100,1)

In [26]:
y_train = 3*np.power(x_train,2)+2+0.2*np.random.rand(x_train.size).reshape(100,1)

In [31]:
model = nn.Linear(1,1)

In [32]:
loss_fn = nn.MSELoss()

In [33]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

In [34]:
for epoch in range(20):
    inputs = torch.from_numpy(x_train).type(dtype)
    targets = torch.from_numpy(y_train).type(dtype)
    
    output = model(inputs)
    loss = loss_fn(output,targets)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    writer.add_scalar('训练损失值',loss,epoch)